In [1]:
import pandas as pd
from google.cloud import storage
import os
import json
import subprocess

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [2]:
import ee
from google.auth.transport.requests import AuthorizedSession
!{'earthengine authenticate'}
#ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

Fetching credentials using gcloud
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=7KEPMwgYkESfKEmYQRssZ0AyXsmKDN&access_type=offline&code_challenge=58YElPlGp8DtgQBcRikKx27HlWpaTmJ9fymLfpHCuhc&code_challenge_method=S256


Credentials saved to file: [/Users/christiannilsen/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update


Successfully saved authorization token.


## Get image information from gcp
Get list of images and their respective uris from the gcp data bucket


In [3]:
def list_blobs_with_prefix(bucket_name, prefix, file_extension='.tif', delimiter=None):
    

    storage_client = storage.Client(project='swhm-prod')

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        if blob.name.endswith(file_extension):
            blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


In [4]:
BUCKET_NAME = 'live_data_layers'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'raster','.tif')

In [5]:
df = pd.DataFrame(blobsout, columns=['blob_name'])
df['uri'] = 'gs://'+BUCKET_NAME+'/'+df['blob_name']
(df)

,blob_name,uri
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif
7,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...
8,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...
9,rasters/Runoff_mm.tif,gs://live_data_layers/rasters/Runoff_mm.tif


## Direct upload to earthengine

In [17]:

file_names = [os.path.splitext(os.path.basename(file_path))[0] for file_path in df['blob_name']]
df["asset_name"] = file_names

pretty_names  = [os.path.splitext(os.path.basename(file_path))[0].replace('_', ' ') for file_path in df['blob_name']]

df["pretty_name"] = pretty_names
df

,blob_name,uri,asset_name,pretty_name
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...,Age_of_Imperviousness,Age of Imperviousness
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...,Flow_Duration_Index,Flow Duration Index
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...,HSPF_Land_Cover_Type,HSPF Land Cover Type
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...,Hydrologic_Response_Units,Hydrologic Response Units
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif,Imperviousness,Imperviousness
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif,Land_Cover,Land Cover
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif,Land_Use,Land Use
7,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...,Population_Density,Population Density
8,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...,Precipitation_mm,Precipitation mm
9,rasters/Runoff_mm.tif,gs://live_data_layers/rasters/Runoff_mm.tif,Runoff_mm,Runoff mm


In [26]:
lay_name = 'Land_Cover'  # replace with the name you're looking for
uri = df.loc[df['asset_name'] == layer_name, 'uri'].values[0]
asset_id = 'projects/ee-swhm/assets/production_layers/' + lay_name
pyramiding_policy = 'mode'

ee_upload = f'''\
earthengine upload image \
--asset_id={asset_id} \
--pyramiding_policy={pyramiding_policy} \
{uri}\
'''
!{ee_upload}

Started upload task with ID: RFJSG6F3M6OJKULNSJRE37C5


earthengine upload image --asset_id=users/username/asset_id --pyramiding_policy=sample

In [7]:
# lay_name = "Land_Cover"
# asset_id = 'projects/ee-swhm/assets/production_layers/' + lay_name
# uri = df.iloc[5].uri
# #check the info 
# rio_cmd = f'rio cogeo info {uri}'
# !{rio_cmd}

Driver: GTiff
File: gs://live_data_layers/rasters/Land_Cover.tif
COG: True
Compression: LZW
ColorSpace: None

Profile
    Width:            396288
    Height:           378880
    Bands:            1
    Tiled:            True
    Dtype:            uint8
    NoData:           None
    Alpha Band:       False
    Internal Mask:    False
    Interleave:       BAND
    ColorMap:         False
    ColorInterp:      ('gray',)
    Scales:           (1.0,)
    Offsets:          (0.0,)

Geo
    Crs:              EPSG:3857
    Origin:           (-13904201.1931867, 6328374.445786312)
    Resolution:       (1.194328566955879, -1.194328566955879)
    BoundingBox:      (-13904201.1931867, 5875867.238338069, -13430903.11404489, 6328374.445786312)
    MinZoom:          6
    MaxZoom:          17

Image Metadata
    AREA_OR_POINT: Area
    OVR_RESAMPLING_ALG: NEAREST

Image Structure
    COMPRESSION: LZW
    INTERLEAVE: BAND
    LAYOUT: COG

Tiling Scheme
    NAME: WEBMERCATORQUAD
    ZOOM_LEVEL: 17



In [11]:
# ee_info = f'earthengine asset info {asset_id}'
# !{ee_info}

{
  "bands": [
    {
      "dataType": {
        "precision": "INT",
        "range": {
          "max": 255
        }
      },
      "grid": {
        "affineTransform": {
          "scaleX": 1.194328566955879,
          "scaleY": -1.194328566955879,
          "translateX": -13904201.1931867,
          "translateY": 6328374.445786312
        },
        "crsCode": "EPSG:3857",
        "dimensions": {
          "height": 378880,
          "width": 396288
        }
      },
      "id": "b1",
      "pyramidingPolicy": "MEAN"
    }
  ],
  "geometry": {
    "coordinates": [
      [
        [
          -124.90357548563415,
          49.31438728345055
        ],
        [
          -124.90357466151073,
          46.592740774523946
        ],
        [
          -124.7457889554153,
          46.59283783034879
        ],
        [
          -124.58800331573232,
          46.59271796025322
        ],
        [
          -124.40531309087523,
          46.592836910698516
        ],
        [
     

NameError: name 'asset_name' is not defined

In [19]:
uri = 'gs://live_data_layers/rasters/Traffic.tif'
pyramiding = 'mean' 
ee_cmd = f'earthengine upload image --asset_id={asset_id} --pyramiding_policy={pyramiding} {uri}'
!{ee_cmd}
#ee_cmd

Started upload task with ID: 6L3T2CW43Q7SK4B5LOYX4UN5


In [12]:
#asset_id = 'projects/ee-swhm/assets/production_layers/' + lay_name
#check the info 
ee_cmd = f'earthengine rm {asset_id}'
ee_cmd
!{ee_cmd}

In [ ]:
# lay_name = "Imperviousness"
# asset_id = 'projects/ee-swhm/assets/production_layers/' + lay_name
# uri = df.iloc[0].uri
# #check the info 
# ee_cmd = f'earthengine rm {asset_id}'
# !{ee_cmd}

In [ ]:
def get_layer_dict(asset): 
    data = json.load(open("data/rasters.json"))
    layer_dict = data[asset]
    print(asset)
    #delete dictionaries 
    try: 
        del layer_dict["layer"]
        del layer_dict["values"]
        del layer_dict["labels"]

    except KeyError:
        pass
    

    layer_dict['pretty_name'] = asset
    return layer_dict

In [ ]:
# Iterate through the rows and upload each blob to Earth Engine
for index, row in df.iterrows():
    if row['blob_name'].endswith('.tif'):
        asset_id = 'projects/ee-swhm/assets/production_layers/' + row['asset_name']
        aname = row['asset_name'] 
        code = f'exports.{aname} = ee.Image("{asset_id}")'
        #print(code)
        
        dcode = f'data.{aname}' 
        print(dcode)

In [ ]:
# Iterate through the rows to generate list for data_raw.js 
for index, row in df.iterrows():
    if row['blob_name'].endswith('.tif'):
        lay_dict = get_layer_dict(row['pretty_name'])
        if (lay_dict['discrete']=='TRUE'):
            pyramiding = 'sample'
        else: 
            pyramiding = 'mean' 
    
        asset_id = 'projects/ee-swhm/assets/production_layers/' + row['asset_name']
        uri = row['uri']
        ee_cmd = f'earthengine upload image --asset_id={asset_id} --pyramiding_policy={pyramiding} {uri}'
        !{ee_cmd}

## Cloud backed assets

## Build the manifest

Create json_data

In [ ]:
df['json_data'] = 'na'
for index, row in df.iterrows():
    key_value = row['pretty_name']
    try: 
        json_value = json.dumps(get_layer_dict(key_value))
    except KeyError:
        json_value = "NA"
    df.at[index, 'json_data'] = json_value


In [ ]:
df

In [ ]:
import requests

def send_request(session, asset_name,props):
    # Earth Engine enabled Cloud Project.
    project_folder = 'ee-swhm'
    # A folder (or ImageCollection) name and the new asset name.
    asset_id = asset_name

    url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId=production_layers/{}'
    #url = 'https://earthengine.googleapis.com/v1beta/projects/{}/assets?assetId=production_layers/{}'
    params = {'overwrite': True}
    response = session.post(
      url = url.format(project_folder, asset_id),
      data = props, 
        params=params
    )
    
    print(json.loads(response.content))
    
    


In [ ]:
import json
import pprint
def process_df(df): 
    for index, row in df.iterrows():
        json_string = row['json_data']
        print(row['pretty_name'])
        print('...')
        if json_string is not None and json_string != 'NA':
            layer = df['pretty_name'][index]
            asset_name = df['asset_name'][index]
            uri = df['uri'][index]

            # Request body as a dictionary.
            request = {
                'type': 'IMAGE',
                'gcs_location': {'uris': uri},
                'properties':json.loads(df['json_data'][index])
            }

            props = json.dumps(request)
            send_request(session, asset_name,props)


## Send the request

Make the POST request to the Earth Engine [`projects.assets.create`](https://developers.google.com/earth-engine/reference/rest/v1beta/projects.assets/create) endpoint.

In [13]:
#process_df(df)

index = 5
layer = df['pretty_name'][index]
asset_name = df['asset_name'][index]
uri = df['uri'][index]
ee_cmd = f'earthengine upload image \
--asset_id=projects/ee-swhm/assets/production_layers/{asset_name} \
{uri}'
print(ee_cmd)


KeyError: 'pretty_name'

In [14]:
df

,blob_name,uri
0,rasters/Age_of_Imperviousness.tif,gs://live_data_layers/rasters/Age_of_Imperviou...
1,rasters/Flow_Duration_Index.tif,gs://live_data_layers/rasters/Flow_Duration_In...
2,rasters/HSPF_Land_Cover_Type.tif,gs://live_data_layers/rasters/HSPF_Land_Cover_...
3,rasters/Hydrologic_Response_Units.tif,gs://live_data_layers/rasters/Hydrologic_Respo...
4,rasters/Imperviousness.tif,gs://live_data_layers/rasters/Imperviousness.tif
5,rasters/Land_Cover.tif,gs://live_data_layers/rasters/Land_Cover.tif
6,rasters/Land_Use.tif,gs://live_data_layers/rasters/Land_Use.tif
7,rasters/Population_Density.tif,gs://live_data_layers/rasters/Population_Densi...
8,rasters/Precipitation_mm.tif,gs://live_data_layers/rasters/Precipitation_mm...
9,rasters/Runoff_mm.tif,gs://live_data_layers/rasters/Runoff_mm.tif


In [ ]:
!{ee_cmd}

In [ ]:
def get_type(file_path): 
    command = ["rio", "cogeo", "info",file_path]
    result = subprocess.run(command, capture_output=True,text=True)
    pattern = r"Dtype:\s+(\w+)"
    # find the Dtype value
    match = re.search(pattern, s)
    if match:
        dtype = match.group(1)
        return(dtype)
    else:
        return("Dtype not found.")


In [ ]:
index = 0
layer = df['pretty_name'][index]
asset_name = df['asset_name'][index]
uri = df['uri'][index]

if (get_type(uri) == 'uint8'): 
    pyramiding_policy = "sample" 
else: 
    pyramiding_policy = "mean"
    
    
    
ee_cmd = f'earthengine upload image \
--asset_id=projects/ee-swhm/assets/production_layers/{asset_name} \
--pyramiding_policy={pyramiding_policy} \
{uri}'
ee_cmd    

In [ ]:
#if need to remove it first 
#ee_cmd = f'earthengine rm projects/ee-swhm/assets/production_layers/Age_of_Imperviousness'

In [ ]:
!{ee_cmd}






In [ ]:
#print(result.stdout)
output_dict = ast.literal_eval(result.stdout)

print(output_dict)


In [ ]:
cmd = f'rio cogeo info {uri}'
!{cmd}

In [ ]:
#direct upload from cmd line 
for index, row in df.iterrows():
    json_string = row['json_data']
    print(row['pretty_name'])
    print('...')
    if json_string is not None and json_string != 'NA':
        layer = df['pretty_name'][index]
        asset_name = df['asset_name'][index]
        uri = df['uri'][index]
    
        

PATCH https://earthengine.googleapis.com/v1/{asset.name=projects/*/assets/**}